In [1]:
%reset
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import glob
import scipy.stats as sci
import math
import statsmodels.stats.multitest as multest
import statsmodels.sandbox.stats.multicomp as mulcomp
import networkx as nx
import gseapy

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Creating directory C:\Users\jmjun\AppData\Local\bioservices\bioservices 


## hypergeomeric tests for the significant TCPs with TSgene, TTD and uniprot

### TSgene

In [9]:
def hyperGeo_TSgene(all_prots, sig_prots, sig_or_ran, pos_or_neg, res_file):
    TSgene=pd.read_table('data/TSgene2.0/Human_TSGs.txt', sep='\t',index_col=0,engine='python')
    TSgene=set(TSgene['GeneSymbol'])
    TSgene=all_prots&TSgene

    p_val = sci.hypergeom.sf(len(sig_prots&TSgene), len(all_prots), len(TSgene), len(sig_prots))
    
    res_file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(sig_or_ran, pos_or_neg, 'TSgene', len(all_prots),
                                                                 len(TSgene), len(sig_prots), len(sig_prots&TSgene), p_val, sig_prots&TSgene))


### TTD

In [10]:
def get_TTD():
    def cancer_related_disease(x):
        code_list=[]
        for code in x.split(', '):
            code_list+=code.split('-')

        for code in code_list:
            if code.startswith('C') or code.startswith('D') :
                return True
        return False

    TTD_to_UPT=pd.read_table('data/TTD/P2-01-TTD_uniprot_all.txt', index_col=0, sep='\t',engine='python')
    TTD_to_UPT=TTD_to_UPT['Uniprot ID'].map(lambda x: x.split(' ')[0]).to_dict()
    UPT_to_gene=pd.read_table('data/TTD/uniprot_to_gene.txt', index_col=0, sep='\t',engine='python')
    UPT_to_gene=UPT_to_gene['To'].to_dict()

    TTD_TarDis=pd.read_table('data/TTD/P1-05-Target_disease.txt', sep='\t',engine='python')
    TTD_TarDis=TTD_TarDis.loc[TTD_TarDis['ICD10'].notnull()]
    TTD_TarDis=TTD_TarDis.loc[TTD_TarDis['ICD10'].map(cancer_related_disease)]

    TTD_available_UPT=set(TTD_TarDis['TTDTargetID'])&set(TTD_to_UPT.keys())

    UPT_list=[]
    for TTD in TTD_available_UPT:
        UPT_list.append(TTD_to_UPT[TTD])

    UPT_available_gene=set(UPT_list)&set(UPT_to_gene.keys())

    gene_list=[]
    for UPT in UPT_available_gene:
        gene_list.append(UPT_to_gene[UPT])

    return set(gene_list)

def hyperGeo_TTD(all_prots, sig_prots, sig_or_ran, pos_or_neg, res_file):
    TTD=get_TTD()
    TTD=all_prots&TTD
    
    p_val = sci.hypergeom.sf(len(sig_prots&TTD), len(all_prots), len(TTD), len(sig_prots))
    
    res_file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(sig_or_ran, pos_or_neg, 'TTD',len(all_prots),
                                                                 len(TTD), len(sig_prots), len(sig_prots&TTD), p_val, sig_prots&TTD))


### uniprot

In [11]:
def get_uniprot(status):
    uniprot=pd.read_table('data/Uniprot/{}_uniprot.txt'.format(status), sep='\t',engine='python')
    uniprot=uniprot.loc[uniprot['Gene names'].notnull()]
    
    uniprot_list=[]
    for ind, row in uniprot.iterrows():
        uniprot_list+=row['Gene names'].split(' ')

    return set(uniprot_list)

def hyperGeo_Uniprot(all_prots, sig_prots, status, sig_or_ran, pos_or_neg, res_file):
    uniprot=get_uniprot(status)
    uniprot=all_prots&uniprot
    
    p_val = sci.hypergeom.sf(len(sig_prots&uniprot), len(all_prots), len(uniprot), len(sig_prots))
    
    res_file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(sig_or_ran, pos_or_neg, 'uniprot({})'.format(status), len(all_prots),
                                                                 len(uniprot), len(sig_prots), len(sig_prots&uniprot), p_val, sig_prots&uniprot))


### constructing a PPI network based on twohybird interactions in BIOGRID database

In [12]:
def get_ppi_nx():
    ppi=pd.read_table('data/BIOGRID/BIOGRID-ORGANISM-Homo_sapiens-3.5.186.tab3.txt', sep='\t')

    ppi_twohyb=ppi.loc[ppi['Experimental System']=='Two-hybrid']
    ppi_twohyb=ppi_twohyb[['Official Symbol Interactor A','Official Symbol Interactor B']]
    ppi_twohyb.columns=['p1','p2']

    # construct ppi network
    ppi_nx=nx.Graph()
    for ind in ppi_twohyb.index:
        p1=ppi_twohyb.loc[ind,'p1']
        p2=ppi_twohyb.loc[ind,'p2']
        ppi_nx.add_edge(p1, p2)
    
    return ppi_nx

### selecting proteins on the paths between the target genes of TF.A and TF. B in the PPI network, where TF.A and TF.B are the significantg TCP

In [13]:
def get_protein_on_path(ppi_nx, g, p, path_len):
    if (g not in ppi_nx.nodes()) or (p not in ppi_nx.nodes()):
        return []
    paths=nx.all_simple_paths(ppi_nx, g, p, cutoff=path_len) # get all simple path between g and p
    protein_list=[]
    for path in paths:
        protein_list+=path[:-1]
    return protein_list

def get_proteins_on_path_between_TCP(sig_or_ran,ppi_nx,neg_or_pos, path_len, sig_cnt_perc):
    sig_TCP=pd.read_table('result/TCP_score_viper/sig_TCP.txt', sep='\t',index_col=0,header=None, engine='python')
    sig_TCP=sig_TCP.loc['both({})'.format(neg_or_pos)].iloc[1].split(',')

    ## randomly selected TCP
    # if sig_or_ran is 'ran', TF pairs are randomly selected as many as the significant TCPs
    if 'ran' in sig_or_ran:
        TF_df=pd.read_table('result/TT_score_viper/TTS_A375.txt', sep='\t',index_col=0,engine='python')
        TF_list=list(TF_df.index)
        ranTF_list=np.random.choice(TF_list, len(sig_TCP)*2, replace=False)
        ranTCP=[]
        for ii in range(int(len(ranTF_list)/2)):
            ranTCP.append(ranTF_list[ii*2]+'|'+ranTF_list[ii*2+1])
        sig_TCP=ranTCP[:]
    print(sig_TCP[:5])
    
    # get proteins on the all paths whose length is lesser than 'path_len' between TF.A and the target genes of TF.B
    tf_target=pd.read_table('data/TRRUST2_TF/trrust2_TF_target_processed.txt', sep='\t',index_col=0)
    all_protein_list=[]
    for p1_p2 in sig_TCP:
        protein_list_for_each_TCP=[]
        p1,p2=p1_p2.split('|')
        for g1 in tf_target.loc[p1,['target']].values.flatten():
            protein_list_for_each_TCP+=get_protein_on_path(ppi_nx, g1, p2, path_len)
        for g2 in tf_target.loc[p2,['target']].values.flatten():
            protein_list_for_each_TCP+=get_protein_on_path(ppi_nx, g2, p1, path_len)
        all_protein_list+=list(set(protein_list_for_each_TCP))
    
    prot_cnt_dic={}
    for prot in set(all_protein_list):
        prot_cnt_dic[prot]=all_protein_list.count(prot)
    
    prot_cnt_sr=pd.Series(prot_cnt_dic)
    sig_prot_sr=prot_cnt_sr.loc[prot_cnt_sr>=(len(sig_TCP)*sig_cnt_perc)]
    
    # return unique proteins on the pahts
    return set(sig_prot_sr.index)

### main

In [14]:
%%time
ppi_nx=get_ppi_nx()
all_prots=set(list(ppi_nx.nodes()))

res_file=open('result/TCP_score_viper/hyperGeo_result.txt','w+')
_=res_file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format('sig_or_ran','sig_TCP_type','database','all_prot','GS','sig_prot','hit','p-value','hit_prot_list'))

path_len=3
sig_cnt_perc=0.2

ran_num=5000

# 1 significant TCPs and 5000 randomly selected TF pairs to compute empirical p-value of the significant TCPs
for sig_or_ran in ['sig']+['ran{}'.format(ii) for ii in range(1,ran_num+1)]:
    print(sig_or_ran)
    sig_pos_prots=get_proteins_on_path_between_TCP(sig_or_ran, ppi_nx, 'pos', path_len, sig_cnt_perc)
    hyperGeo_TSgene(all_prots, sig_pos_prots, sig_or_ran, 'pos', res_file)
    hyperGeo_Uniprot(all_prots, sig_pos_prots, 'tumor_suppressor', sig_or_ran, 'pos', res_file)
    
    sig_neg_prots=get_proteins_on_path_between_TCP(sig_or_ran, ppi_nx, 'neg', path_len, sig_cnt_perc)
    hyperGeo_TTD(all_prots, sig_neg_prots, sig_or_ran, 'neg', res_file)
    hyperGeo_Uniprot(all_prots, sig_neg_prots, 'oncogene', sig_or_ran, 'neg', res_file)
    
res_file.close()

sig
['IFI16|MXI1', 'POU2F2|POU4F1', 'NFIL3|POU2F2', 'FOXP3|NFATC3', 'BACH2|PGR']
['ENO1|HOXA1', 'SFPQ|SPDEF', 'E2F6|TFDP1', 'AATF|ENO1', 'FOXL2|SOX17']
ran1
['KLF8|TRIM16', 'RBPJ|THRB', 'LDB1|TCF7L2', 'EAPP|NCOA2', 'KLF6|ERG']
['STAT4|AIP', 'TBX21|ISL1', 'FOXG1|ELF2', 'GFI1B|KLF6', 'ZNF202|KDM4C']
ran2
['OTX2|USF2', 'ZNF224|UHRF1', 'PIAS1|ID4', 'TOP2B|GCM1', 'NR3C1|TCF7L2']
['BARD1|DNMT3L', 'HIC1|CREB3L1', 'NFYA|NFIB', 'FOSL2|ZNF382', 'GTF3A|FOXF1']
ran3
['HOXA7|MYOD1', 'ANKRD1|HMGA2', 'DUX4|TBX5', 'HSF2|HINFP', 'NFYC|SP7']
['PAX2|KLF5', 'ID4|FOXI1', 'MTF1|CBX8', 'SOX6|MECP2', 'NANOG|PITX1']
ran4
['TRAF6|FOXP3', 'HINFP|POU5F1', 'CREBBP|HNF4G', 'ONECUT2|KAT5', 'CBFA2T3|RUNX2']
['HDAC11|LMO4', 'SPDEF|VHL', 'CTBP1|SOX6', 'SALL3|COPS5', 'SOX17|HNF4G']
ran5
['PROX1|CEBPA', 'MTA3|ZBTB5', 'ESR2|RELB', 'TCF3|ATOH1', 'MZF1|CDX2']
['MXD1|HHEX', 'DAXX|ERF', 'MED23|UHRF1', 'ASCL1|SALL4', 'SATB1|MTA1']
Wall time: 2min 27s


## biological process enrichment for TFs in BTCPs

In [2]:
def get_sig_TCP():
    sig_TCP=pd.read_table('result/TCP_score_viper/sig_TCP.txt', sep='\t',index_col=0,header=None)
    
    TF_list=[]
    
    pos_sig_TCP=sig_TCP.loc['both(pos)'].iloc[1]
    for TF_pair in pos_sig_TCP.split(','):
        TF_list+=TF_pair.split('|')
        
    neg_sig_TCP=sig_TCP.loc['both(neg)'].iloc[1]
    for TF_pair in neg_sig_TCP.split(','):
        TF_list+=TF_pair.split('|')
        
    return list(set(TF_list))

TF_list=get_sig_TCP()
print('# of TF in the sig TCPs: ',len(TF_list))

gseapy.enrichr(gene_list=TF_list, gene_sets='KEGG_2019_Human',            outdir='result/TCP_score_viper/pathway_enrichment/')
gseapy.enrichr(gene_list=TF_list, gene_sets='GO_Biological_Process_2018', outdir='result/TCP_score_viper/pathway_enrichment/')

# of TF in the sig TCPs:  147
